In [10]:
#한국어 영어 번역 데이터셋 만들기
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import unicodedata
import re

# 실제 프로젝트 AI hub 기타등등..

korean_sentences = [
    "안녕하세요",
    "오늘 날씨가 좋아요",
    "드라마 보고 있어",
    "이것은 사과입니다",
    "고양이가 자고 있어요",
    "내일 비가 올까요",
    "저는 커피를 좋아해요",
    "그는 의사입니다",
    "이 책은 재미있어요",
    "오늘 몸이 안 좋아"
]

english_sentences = [
    "hello",
    "the weather is nice today",
    "I’m watching a drama",
    "this is an apple",
    "the cat is sleeping",
    "will it rain tomorrow",
    "i like coffee",
    "he is a doctor",
    "this book is interesting",
    "I don’t feel well today"
]

# 텍스트 전처리
# uicode 정규화 , 툭수문자처리
# NFD Normalization Form Decomposition -> 분해 가능한 모든 문자를 분해한다.
unicodedata.normalize('NFD',"e ' ")

"e ' "

$\acute{e}$

단일문자 U+00E9

$e$ + $\acute{}$

$e$ + 악센트가 결합이 된 형태

Unicode 정규화란 ?  같은 문장처럼 보이지만 내부적으로 다른 바이트 조합을 가지는 경우가 있는데 이걸 동일한 내부 표현으로 바꿔주는 과정

In [17]:
def preprocess_sentence(sentence , is_korean = False):
    '''
    Unicode 정규화 , 특수문자 처리
    Args : 
        sentence : 원본
        is_korean : 한국어 여부
    return:
        전처리된 문장
    '''
    sentence = unicodedata.normalize('NFD' , sentence)
    if not is_korean:
        sentence = sentence.lower()
    sentence = sentence.strip() # .strip() 좌우공백제거
    # 안녕하세요! -> 안녕하세요 !  정규식으로 특수문자 전 후에 공백 추가
    # r"[?.!,]"   문자 중에 ?.!, 특수문자가 나오면 문자를 그룹으로 캡쳐
    # r"\1"  캡쳐한 문자 (\1) 앞뒤로 공백을 하나씩 넣는다.
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]', " ", sentence)
    # 시작 종료 토큰 추가
    sentence = '<start>' + sentence + '<end>'
    return sentence

korean_processed = [preprocess_sentence(kor) for kor in korean_sentences]
english_processed = [preprocess_sentence(eng) for eng in english_sentences]

# 문장을 정수 시퀀스로 변환
# 3 토크나이저

def create_tokenizer(sentence):
    '''
    단어를 정수 인덱스로 변환
    Vocabulary 구축 , word to index mapping
    Args:
        sentence : 문장 리스트
    returns:
        Tokenizer : keras Tokenizer 객체
    '''
    tokenizer = Tokenizer(
        filters='' , # 필터 비활성화
        oov_token = '<unk>'
    )
    # 모든 문장으로 단어사전 구축
    tokenizer.fit_on_texts(sentence)
    return tokenizer

# 한국어 영어 각가의 토크나이져 생성
korean_tokenizer = create_tokenizer(korean_processed)
english_tokenizer = create_tokenizer(english_processed)

# 단어사전의 크기를 확인
korean_vocab_size = len(korean_tokenizer.word_index)+1 # oov_token = '<unk>' 때문에 플러스 1을 해준거임
english_vocab_size = len(english_tokenizer.word_index)+1
print(f'한국어 사전 크기 : {korean_vocab_size}')
print(f'영어 사전 크기 : {english_vocab_size}')

# 정수 시퀀스 변경
def encode_sentences(tokenizer , sentneces , maxlen):
    '''
    문장을 고정 길이의 정수 시퀀스로 변환
    padding : 짧은 문장을 동일 길이로 맞추는
    Args : 
        Tokenizer
        sentence : 문장리스트
        maxlen : 가장 긴 문장
    returns:
        패딩된 정수 시퀀스 배열 tf의 pad_sequence
    '''
    sequence = tokenizer.texts_to_sequences(sentneces) # [i love you] -> [1,5,7]
    
    # 길이맞추기 (패딩추가)
    padded = tf.keras.preprocessing.sequence.pad_sequences(
        sequence, 
        maxlen = maxlen,
        padding='post'
    )
    return padded

# 최대 시퀀스의 길이 결정 (가장 긴 문장을 기준으로!)
max_korean_len = max( len(s.split()) for s in korean_processed )
max_english_len = max( len(s.split()) for s in english_processed )
print(f'시퀀스의 최대길이')
print(f'한국어 : {max_korean_len}')
print(f'영어 : {max_english_len}')

# 인코딩 수행
korean_tensor = encode_sentences(korean_tokenizer , korean_processed , max_korean_len)
english_tensor = encode_sentences(english_tokenizer , english_processed , max_english_len)
print(f'인코딩 두번째 문장')

print(f'한국어 두번째 문장 : {korean_processed[1]}')
print(f'한국어 인코딩 문장 : {korean_tensor[1]}')

print(f'영어 두번째 문장 : {english_processed[1]}')
print(f'영어 인코딩 문장 : {english_tensor[1]}')

한국어 사전 크기 : 28
영어 사전 크기 : 31
시퀀스의 최대길이
한국어 : 4
영어 : 5
인코딩 두번째 문장
한국어 두번째 문장 : <start>오늘 날씨가 좋아요<end>
한국어 인코딩 문장 : [2 4 5 0]
영어 두번째 문장 : <start>the weather is nice today<end>
영어 인코딩 문장 : [ 3  9  2 10  4]
